In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from pylab import cm
import os

%matplotlib inline

In [ ]:
parent_dir = os.path.split(os.getcwd())[0]

In [ ]:
landuse = pd.read_csv(parent_dir + '\\Data\\New\\landuse_lms.csv')
landuse_orig = pd.read_csv(parent_dir + '\\Data\\Wijk buurt\\85217NED_UntypedDataSet_28032024_162743.csv', delimiter=';')

In [ ]:
zones = gpd.read_file(parent_dir + '\\Data\\New\\lms_zone_du_new.shp')

In [ ]:
landuse_area = pd.read_csv(parent_dir + '\\Data\\New\\landuse_lms_area.csv')


Get some number on total land-use to see if it is significant

In [ ]:
nl_tot = landuse_orig.iloc[0]
tot_area = nl_tot.TotaleOppervlakte_4
landuse_orig.iloc[0:1].columns

In [ ]:
residential = nl_tot.Woonterrein_10
print(f'Tot area: {residential}, ratio: {residential / tot_area * 100:.2f}%')

In [ ]:
services = nl_tot.TerreinVoorDetailhandelEnHoreca_11 + nl_tot.TerreinVoorOpenbareVoorzieningen_12 + nl_tot.TerreinVoorSociaalCultureleVoorz_13
print(f'Tot area: {services}, ratio: {services / tot_area * 100:.2f}%')

In [ ]:
industrial = nl_tot.Bedrijventerrein_14 + nl_tot.TotaalSemiBebouwdTerrein_15 - nl_tot.Begraafplaats_18
print(f'Tot area: {industrial}, ratio: {industrial / tot_area * 100:.2f}%')

In [ ]:
recr_green_water = (nl_tot.TotaalRecreatieterrein_22 + nl_tot.TotaalBosEnOpenNatuurlijkTerrein_31 
                    + nl_tot.TotaalBinnenwater_35 + nl_tot.TotaalBuitenwater_45 + nl_tot.Begraafplaats_18)
print(f'Tot area: {recr_green_water}, ratio: {recr_green_water / tot_area * 100:.2f}%')                                                           

In [ ]:
argiculture = nl_tot.TotaalAgrarischTerrein_28
print(f'Tot area: {argiculture}, ratio: {argiculture / tot_area * 100:.2f}%')    

In [ ]:
infra = nl_tot.TotaalVerkeersterrein_5
print(f'Tot area: {infra}, ratio: {infra / tot_area * 100:.2f}%')    


In [ ]:
residential + services + industrial + recr_green_water + argiculture + infra

In [ ]:
landuse['Residential'] = landuse.Woonterrein_10
landuse['Services'] = (landuse.TerreinVoorDetailhandelEnHoreca_11 
                       + landuse.TerreinVoorOpenbareVoorzieningen_12 
                       + landuse.TerreinVoorSociaalCultureleVoorz_13)
landuse['Industrial'] = landuse.Bedrijventerrein_14 + landuse.TotaalSemiBebouwdTerrein_15 - landuse.Begraafplaats_18
landuse['Nature'] = (landuse.TotaalRecreatieterrein_22 + landuse.TotaalBosEnOpenNatuurlijkTerrein_31 
                    + landuse.TotaalBinnenwater_35 + landuse.TotaalBuitenwater_45 + landuse.Begraafplaats_18)
landuse['Agricultural'] = landuse.TotaalAgrarischTerrein_28
landuse['Infra'] = landuse.TotaalVerkeersterrein_5

In [ ]:
landuse['Residential_area'] = landuse_area.Woonterrein_10
landuse['Services_area'] = (landuse_area.TerreinVoorDetailhandelEnHoreca_11 
                       + landuse_area.TerreinVoorOpenbareVoorzieningen_12 
                       + landuse_area.TerreinVoorSociaalCultureleVoorz_13)
landuse['Industrial_area'] = landuse_area.Bedrijventerrein_14 + landuse_area.TotaalSemiBebouwdTerrein_15 - landuse_area.Begraafplaats_18
landuse['Nature_area'] = (landuse_area.TotaalRecreatieterrein_22 + landuse_area.TotaalBosEnOpenNatuurlijkTerrein_31 
                    + landuse_area.TotaalBinnenwater_35 + landuse_area.TotaalBuitenwater_45 + landuse_area.Begraafplaats_18)
landuse['Agricultural_area'] = landuse_area.TotaalAgrarischTerrein_28
landuse['Infra_area'] = landuse_area.TotaalVerkeersterrein_5

In [ ]:
tot_area_arr = np.array([residential, services, industrial, recr_green_water, argiculture, infra]) / tot_area

In [ ]:
landuse

In [ ]:
labels = ['Residential', 'Services', 'Industrial',
          'Nature', 'Agricultural', 'Infra']

In [ ]:
e = np.zeros(len(landuse))
s = np.zeros(len(landuse))

for i in range(6):
    e_j = np.round((landuse[labels[i]] * np.log(landuse[labels[i]])) / np.log(6), 3)
    e_j[e_j != e_j] = 0
    e -= e_j

    nsi_j = landuse[labels[i] + '_area'] / landuse[labels[i] + '_area'].sum()
    landuse[labels[i] + '_nsi'] = nsi_j

    s += np.abs(landuse[labels[i]] - tot_area_arr[i])

landuse['Entropy'] = e

In [ ]:
landuse['Special'] = np.round(s * 0.5, 3)

In [ ]:
landuse[labels[5] + '_area'].sum()

In [ ]:
landuse

## Make some plots to observe indices

In [ ]:
zone_landuse = zones.merge(landuse, on='ZONE_ID')

Entropy

In [ ]:
f, ax = plt.subplots(1, 2)
f.set_figwidth(13)
f.set_figheight(6)

cmap = cm.get_cmap('OrRd')
# cmap_cbs = cm.get_cmap('YlOrRd_r', 5)

zone_landuse.plot(ax=ax[0], column='Entropy', cmap=cmap, vmax=1, vmin=0,
         legend=True, legend_kwds={"label": "Entropy measure"})

zone_landuse.plot(ax=ax[1], column='Special', legend=True, cmap=cmap,
         legend_kwds={"label": "National specialisation index"})

ax[0].set_title('Entropy measure')
ax[1].set_title('National specialisation index')

ax[0].set_facecolor('lightskyblue')
ax[1].set_facecolor('lightskyblue')

In [ ]:
landuse_index = landuse[['ZONE_ID', 'Residential', 'Services', 'Industrial', 'Nature',
       'Agricultural', 'Infra', 'Entropy', 'Special']]

landuse_index

## Save to csv

In [ ]:
landuse_index.to_csv(parent_dir + '\\Data\\New\\lms_diversity_lu.csv')